# <center>Проект 2 </br>Подгрузка новых данных. Уточнение анализа</center>

## Оглавление:

* [Знакомство с данными](#Знакомство_с_данными)
* [Предварительный анализ данных](#Предварительный-анализ-данных)
    * [Задание 2.1](#Задание-2.1)
    * [Задание 2.2](#Задание-2.2)
    * [Задание 2.3](#Задание-2.3)
    * [Задание 2.4](#Задание-2.4)
* [Глобальный анализ показателей](#Глобальный-анализ-показателей)
    * [Задание 3.1](#Задание-3.1)
    * [Задание 3.2](#Задание-3.2)
    * [Задание 3.3](#Задание-3.3)
    * [Задание 3.4](#Задание-3.4)
    * [Задание 3.5](#Задание-3.5)
* [Анализ кандидатов для заказчиков](#Анализ-кандидатов-для-заказчиков)
    * [Задание 4.1](#Задание-4.1)
    * [Задание 4.2](#Задание-4.2)
* [Общий вывод по результатам анализа рынка труда](#Общий-вывод-по-результатам-анализа-рынка-труда)

К сожалению, ссылки внутри ноутбука не работают на GitHub. Необходимо либо скачать ноутбук, либо перейти по [ссылке](https://nbviewer.org/github/DS-Makovetskiy/DS/blob/main/Project_2_Loading_new_data._Refining_the_analysis/Project_2_Loading_new_data._Refining_the_analysis.ipynb).

## Знакомство с данными

Структура базы данных **project_sql**.

<center><IMG src='./images/SQL_pj2_2_1.png' width=400></center>

Познакомимся с каждой таблицей.

**VACANCIES**

Таблица хранит в себе данные по вакансиям и содержит следующие столбцы:

<center><IMG src='./images/SQL_pj2_2_2.png' width=600></center>

Зарплатная вилка — это верхняя и нижняя граница оплаты труда в рублях (зарплаты в других валютах уже переведены в рубли). Соискателям она показывает, в каком диапазоне компания готова платить сотруднику на этой должности.

**AREAS**

Таблица-справочник, которая хранит код региона и его название.

<center><IMG src='./images/SQL_pj2_2_3.png' width=600></center>

**EMPLOYERS**

Таблица-справочник со списком работодателей.

<center><IMG src='./images/SQL_pj2_2_4.png' width=600></center>

**INDUSTRIES**

Таблица-справочник вариантов сфер деятельности работодателей.

<center><IMG src='./images/SQL_pj2_2_5.png' width=600></center>

**EMPLOYERS_INDUSTRIES**

Дополнительная таблица, которая существует для организации связи между работодателями и сферами их деятельности.

Эта таблица нужна нам, поскольку у одного работодателя может быть несколько сфер деятельности (или работодатели могут вовсе не указать их). Для удобства анализа необходимо хранить запись по каждой сфере каждого работодателя в отдельной строке таблицы.

<center><IMG src='./images/SQL_pj2_2_6.png' width=600></center>

## Предварительный анализ данных

### Задание 2.1

Рассчитайте максимальный возраст кандидата в таблице.

```SQL
SELECT
    MAX(age) AS max_age -- Запрос на вывод максимального возраста среди кандидатов
FROM
    hh.candidate
```

**Результат:**

max_age|
---|
100|

**Вывод:**  
Соискатель, желающий найти работу в возрасте 100-а лет, выглядит по меньшей мере странно и можно сделать вывод о том, что при заполнении резюме, человек допустил ошибку в указании текущего возраста.

### Задание 2.2

Теперь давайте рассчитаем минимальный возраст кандидата в таблице.

```SQL
SELECT
    MIN(age) AS min_age -- Запрос на вывод минимального возраста среди кандидатов
FROM
    hh.candidate
```

**Результат:**

min_age|
---|
14|

**Вывод:**  
Поиск работы в 14-ти летнем возрасте согласно законодательства РФ возможен только в свободное от учебы время. Если выполнить более расширенный запрос, то окажется что соискатель 14-ти летнего возраста в данный момент занимает должность “Ведущий инженер-программист” и рассчитывает на “полный рабочий день” при желаемой зарплате в 60 тыс. р., что выглядит очень неправдоподобно и по всей видимости является ошибкой или намеренным искажением данных.

### Задание 2.3

Попробуем «почистить» данные. Напишите запрос, который позволит посчитать для каждого возраста, сколько человек этого возраста у нас есть.  
Отсортируйте результат по возрасту в обратном порядке.

```SQL
SELECT
    age,
    COUNT(id) AS cnt -- Подсчет количества соискателей каждого возраста
FROM
    hh.candidate
GROUP BY age -- Группировка количества соискателей по возрасту
ORDER BY age DESC -- Сортировка по возрасту в обратном порядке
```

**Результат:**

age|cnt
---|---
100|1
77|1
76|1
---|---
16|4
15|2
14|1

**Вывод:**  
Из таблицы видно, что основной возраст соискателей сосредоточен в диапазоне от 20 до 45 лет. С увеличением количества резюме до 30 лет и постепенным спадом размещаемых резюме с дальнейшим увеличением возраста кандидатов. Это может быть обусловлено более активным поиском лучшего места работы до 30 лет, после чего люди склонны оставаться в текущих кампаниях и расти уже в должности, не меняя саму компанию. Либо искать новое место работы по рекомендациям, без размещения резюме.  

Возраст в 100 лет встречается всего один раз, с большим отрывом от общего распределения возраста среди всех кандидатов. Данный показатель является ошибкой и его необходимо исключить из дальнейшего анализа, чтобы не исказить будущую модель.  

Следует более пристально проанализировать соискателей в возрасте до 18 лет с целью исключить из анализа объективно неправдоподобные резюме, которые могут исказить анализ. Например если в этом возрасте соискатели рассматривают полную занятость на полный рабочий день (учебу никто не отменял) или имеют текущую должность не начального уровня, а более высокой квалификации, что объективно невозможно в раннем возрасте.

### Задание 2.4

По данным Росстата, средний возраст занятых в экономике России составляет 39.7 лет. Мы округлим это значение до 40. Найдите количество кандидатов, которые старше данного возраста. Не забудьте отфильтровать «ошибочный» возраст 100.

```SQL
SELECT
    COUNT(age) AS cnt -- Подсчет количества кандидатов
FROM
    hh.candidate
WHERE
    age > 40 AND age != 100 -- Отбор кандидатов старше 40 лет, с исключением ошибочного возраста в 100 лет
```

**Результат:**

cnt|
---|
6 263|

**Вывод:**  
Полученный результат в 6263 кандидата составляет приблизительно 14% от общего числа соискателей (44744 человек). Можно сделать вывод о том, что подавляющее большинство резюме размещено в более молодом возрасте, до 40-а лет. Это наблюдение подтверждает предыдущая выборка количества соискателей каждого возраста.  

Результат запроса также подтверждает средний возраст занятых в экономике России, так как в этом возрасте люди как правило имеют стабильный доход на постоянном месте работы, о чем свидетельствует снижение количества размещений резюме после 40-а лет. 

[&#8679; к оглавлению](#Оглавление:)

## Глобальный анализ показателей

### Задание 3.1

Для начала напишите запрос, который позволит узнать, сколько у нас кандидатов из каждого города.  
Формат выборки: city, cnt.  
Группировку таблицы необходимо провести по столбцу title, результат отсортируйте по количеству в обратном порядке.

```SQL
SELECT
    c.title,
    COUNT(cand.id) AS cnt -- Подсчет количества кандидатов в каждом городе
FROM
    hh.candidate AS cand
    JOIN hh.city AS c ON cand.city_id = c.id
GROUP BY c.title -- Группировка по городу проживания
ORDER BY cnt DESC -- Сортировка по количеству в обратном порядке
```

**Результат:**

title|cnt
---|---
Москва|16 622
Санкт-Петербург|4 937
Краснодар|1 066
...|...
Черняховск|1
Магас|1
Итатский|1

**Вывод:**  
Из таблицы видно, что наибольшее количество резюме расположено в крупных городах, с большим отрывом Санкт-Петербурга и Москвы. Это может быть обусловлено как большим количеством жителей в крупных городах, так и большим сосредоточением различных компаний на город, особенно в Москве. Это также может повлиять на желание кандидатов переехать из небольших городов в более крупные.  

В небольших городах, с малым количеством размещенных резюме, по всей видимости люди не склонны так часто менять место работы, либо существует буквально несколько градообразующих предприятий, на которых и работает большая часть населения этого города.

### Задание 3.2

Москва бросается в глаза как, пожалуй, самый активный рынок труда. Напишите запрос, который позволит понять, каких кандидатов из Москвы устроит «проектная работа».  
Формат выборки: gender, age, desirable_occupation, city, employment_type.  
Отсортируйте результат по id кандидата.

```SQL
SELECT
    cand.gender,
    cand.age,
    cand.desirable_occupation,
    c.title AS city,
    cand.employment_type
FROM
    hh.candidate AS cand
    JOIN hh.city as c ON c.id = cand.city_id
WHERE
    c.title = 'Москва' AND cand.employment_type LIKE '%проектная работа%' -- Отбор кандидатов из Москвы, готовых на проектную работу
ORDER BY cand.id -- Сортировка по ID кандидата
```

**Результат:**

gender|age|desirable_occupation|city|employment_type
---|---|---|---|---
M|38|Веб-разработчик (HTML / CSS / JS / PHP / базы данных; фреймворки, дизайн, интерфейсы, CMS)|Москва|частичная занятость, проектная работа, полная занятость
M|31|Специалист|Москва|частичная занятость, проектная работа, полная занятость
F|42|pre-sale инженер, pre-sale менеджер|Москва|частичная занятость, проектная работа, полная занятость
---|---|---|---|---
F|25|Email-маркетолог, Автор|Москва|проектная работа, полная занятость
M|26|Начальник отдела|Москва|стажировка, волонтерство, частичная занятость, проектная работа, полная занятость
M|41|Программист микроконтроллеров, Altium Designer|Москва|проектная работа, частичная занятость, полная занятость

**Вывод:**  
Исходя из полученных данных можно сделать вывод о том, что в Москве, на проектную работу готовы те соискатели, которые рассматривают должности преимущественно в IT сфере, к тому же наряду с проектной работой чаще рассматривают и частичную занятость. Это может свидетельствовать о желании работать удаленно или в более свободном графике, к примеру для совмещения нескольких должностей\мест работы.  

Средний возраст таких кандидатов равен 32-м годам.  

Соотношение мужчин и женщин составляет 84% и 16% соответственно. Из чего можно сделать вывод, что проектную работу в IT сфере значительно чаще выбирают мужчины.  

Количество соискателей рассматривающих проектную работу, составляет приблизительно 18% от общего числа размещенных резюме в г. Москва. Что подтверждает наблюдение о том, что в Москве расположено большее количество крупных компаний, с вакансиями более высокой квалификации чем в регионах.

### Задание 3.3

Данных оказалось многовато. Отфильтруйте только самые популярные IT-профессии — разработчик, аналитик, программист.  
Обратите внимание, что данные названия могут быть написаны как с большой, так и с маленькой буквы.  
Отсортируйте результат по id кандидата.

```SQL
SELECT
    cand.gender,
    cand.age,
    cand.desirable_occupation,
    c.title AS city,
    cand.employment_type
FROM
    hh.candidate AS cand
    JOIN hh.city as c ON c.id = cand.city_id
WHERE
    c.title = 'Москва' -- Отбор кандидатов из Москвы, готовых на проектную работу, среди профессий: Разработчик, Аналитик и Программист
    AND cand.employment_type LIKE '%проектная работа%'
    AND cand.desirable_occupation ILIKE ANY (ARRAY['%разработчик%', '%аналитик%', '%программист%'])
ORDER BY cand.id -- Сортировка по ID кандидата
```

**Результат:**

gender|age|desirable_occupation|city|employment_type
---|---|---|---|---
M|38|Веб-разработчик (HTML / CSS / JS / PHP / базы данных; фреймворки, дизайн, интерфейсы, CMS)|Москва|частичная занятость, проектная работа, полная занятость
M|22|Программист С++|Москва|проектная работа, частичная занятость
M|25|Frontend-разработчик|Москва|стажировка, волонтерство, частичная занятость, проектная работа, полная занятость
---|---|---|---|---
M|30|Инженер-программист|Москва|стажировка, частичная занятость, проектная работа, полная занятость
M|22|Программист .NET junior|Москва|частичная занятость, проектная работа, полная занятость
M|41|Программист микроконтроллеров, Altium Designer|Москва|проектная работа, частичная занятость, полная занятость

**Вывод:**  
Отфильтровав самые популярные IT профессии из предыдущей выборки, данных осталось значительно меньше, всего 778 результатов, что составляет 4% от общего числа размещенных резюме в г. Москва.  

В целом, наблюдения совпадают с предыдущим запросом, соискатели также в большинстве случаев рассматривают проектную работу наряду с частичной занятостью и средний возраст кандидатов равен 33 годам.  

Соотношение мужчин и женщин также схоже с предыдущим запросом и составляет 89% и 11% соответственно. Что также свидетельствует о том, что в IT профессиях преобладают мужчины. 

### Задание 3.4

Для общей информации попробуйте выбрать номера и города кандидатов, у которых занимаемая должность совпадает с желаемой.  
Формат выборки: id, city.  

Отсортируйте результат по городу и id кандидата.

```SQL
SELECT
    cand.id,
    c.title AS city
FROM
    hh.candidate AS cand
    JOIN hh.city as c ON c.id = cand.city_id
WHERE
    cand.desirable_occupation = cand.current_occupation -- Фильтрация кандидатов, чья желаемая и текущая должность совпадают
ORDER BY city, cand.id -- Сортировка результата по названию города и ID кандидата
```

**Результат:**

id|city
---|---
2 009|Абакан
10 340|Абакан
14 449|Абакан
---|---
41 970|Ярославль
1 462|Ясногорск
9 701|Яхрома

**Вывод:**  
Из полученных данных можно заметить следующую закономерность - чем больше город, тем большее количество соискателей ищут работу с той же должностью что и занимают. Это может говорить о том, что кандидат в целом доволен своей текущей работой, но по какой то причине желает сменить место работы, возможно с целью повышения зарплаты или желает устроиться в более крупную компанию с возможностью карьерного роста в данном направлении. В остальных городах количество кандидатов с совпадением должностей невелико, что говорит о большей подвижности рынка труда и миграции специалистов с одной должности на другую.  

Большее число совпадений должностей наблюдается в Москве и Санкт-Петербурге.  

При этом общее количество соискателей с совпадающей текущей и желаемой должностью составляет всего 11.4% от общего числа резюме. Это говорит о том, что в большинстве случаев кандидаты всё же склонны менять должность (например повышение в должности) или сферу деятельности, осваивая новые профессии.

### Задание 3.5

Определите количество кандидатов пенсионного возраста.  
Пенсионный возраст для мужчин наступает в 65 лет, для женщин — в 60 лет.

```SQL
SELECT
    count(*) -- Подсчет общего количества кандидатов
FROM
    hh.candidate
WHERE
    (age >= 60 AND gender = 'F' -- Отбор кандидатов пенсионного возраста среди женщин и мужчин
    OR age >= 65 AND gender = 'M')
    AND age != 100 -- Без учета ошибочного 100 летнего возраста
```

**Результат:**

count|
---|
75|

**Вывод:**  
Из всего набора данных (44744 кандидатов), всего 75 (0,2% от общего числа) человек пенсионного возраста рассматривают возможность трудоустройства. Что логично, так как большинство людей в этом возрасте на самом деле выходят на пенсию, либо с учетом большого опыта работы двигаются вверх по карьерной лестнице внутри одной компании (резюме размещать нет необходимости), либо склонны менять место работы по рекомендациям или приглашению от других компаний.

[&#8679; к оглавлению](#Оглавление:)

## Анализ кандидатов для заказчиков

### Задание 4.1

Для добывающей компании нам необходимо подобрать кандидатов из Новосибирска, Омска, Томска и Тюмени, которые готовы работать вахтовым методом.  
Формат выборки: gender, age, desirable_occupation, city, employment_type, timetable_type.  
Отсортируйте результат по городу и номеру кандидата.

```SQL
SELECT
    cand.gender,
    cand.age,
    cand.desirable_occupation,
    c.title AS city,
    cand.employment_type,
    tt.title AS timetable_type
FROM
    hh.candidate AS cand
    JOIN hh.city AS c ON c.id = cand.city_id
    JOIN hh.candidate_timetable_type AS ctt ON ctt.candidate_id = cand.id
    JOIN hh.timetable_type AS tt ON tt.id = ctt.timetable_id
WHERE -- Отбор кандидатов из городов Новосибирск, Омск, Томск и Тюмень, готовых работать вахтовым методом
    c.title IN ('Новосибирск', 'Омск', 'Томск', 'Тюмень') AND (tt.title = 'вахтовый метод')
ORDER BY c.title, cand.id -- Сортировка по городу и ID кандидата
```

**Результат:**

gender|age|desirable_occupation|city|employment_type|timetable_type
---|---|---|---|---|---
M|29|ИТ Инженер|Новосибирск|полная занятость|вахтовый метод
M|25|Заместитель начальника лаборатории|Новосибирск|проектная работа, стажировка, частичная занятость, полная занятость|вахтовый метод
M|30|Ведущий инженер, Специалист по защите информации,|Новосибирск|частичная занятость, полная занятость|вахтовый метод
---|---|---|---|---|---
M|42|Инженер|Томск|проектная работа, частичная занятость, полная занятость|вахтовый метод
M|31|Инженер связи|Тюмень|полная занятость|вахтовый метод
M|31|Инженер АСУ ТП, АСУ, Мастер КИП, Программист АСУ|Тюмень|полная занятость|вахтовый метод

**Вывод:**  
Сразу бросается в глаза крайне небольшое количество кандидатов, всего 11 человек, при этом интересно, что все мужчины (можно предположить, что женщины крайне редко рассматривают работу вахтовым методом). Средний возраст всех соискателей составляет 30 лет. Как правило к 30 годам человек набирается достаточно опыта, может поработать несколько лет вахтовым методом получая более высокую зарплату, а затем, накопив определенную сумму денег для переезда или покупки квартиры переходить на обычную, работу в городе или на местном заводе.

### Задание 4.2

Для заказчиков из Санкт-Петербурга нам необходимо собрать список из 10 желаемых профессий кандидатов из того же города от 16 до 21 года (в выборку включается 16 и 21, сортировка производится по возрасту) с указанием их возраста, а также добавить строку Total с общим количеством таких кандидатов. Напишите запрос, который позволит получить выборку вида:

```SQL
(SELECT
    cand.desirable_occupation,
    cand.age
FROM
    hh.candidate AS cand
    JOIN hh.city AS c ON c.id = cand.city_id
WHERE
    c.title = 'Санкт-Петербург' AND age BETWEEN 16 AND 21 -- Отбор кандидатов из Санкт-Петербурга, в возрасте от 16 до 21 года
ORDER BY cand.age
LIMIT 10) -- Выводим на экран первые 10 значений

UNION ALL

SELECT
    'Total', COUNT(*) -- Добавляем итоговое, общее количество соискателей в конце таблицы
FROM
    hh.candidate AS cand
    JOIN hh.city AS c ON c.id = cand.city_id
WHERE
    c.title = 'Санкт-Петербург' AND age BETWEEN 16 AND 21
```

**Результат:**

desirable_occupation|age
---|---
Системный администратор|16
Junior Разработчик C++/C#|18
Программист|18
---|---
3D-дизайнер|18
Java-разработчик|18
Total|161

**Вывод:**  
Как видно из полученной таблицы, в Санкт-Петербурге, молодые кандидаты, в возрасте от 16 до 21 года в подавляющем большинстве предпочитают IT профессии, как наиболее перспективное направление для начинающих специалистов. Это может говорить о высоком качестве образования в университетах города с более широким набором специальностей в IT сфере.

## Общий вывод по результатам анализа рынка труда

1. Предварительный анализ данных показывает, что люди склонны допускать ошибки во время заполнения резюме или по какой-то причине намеренно искажать данные. Их следует находить и исключать из анализа.  
В молодом возрасте, начиная от 20 лет, соискатели значительно чаще меняют место работы в поисках наиболее подходящей компании и должности, с последующим ростом в должности или переходах в смежные отделы уже внутри одной компании, это отчасти может объяснить спад количества резюме с увеличением возраста кандидата. Наибольшее количество кандидатов сосредоточено в возрасте от 20 до 45 лет, с пиком в 30 лет.
2. Глобальный анализ показателей хорошо демонстрирует зависимость размера города и количества размещенных в нем резюме, чем больше город, тем больше кандидатов ищут работу, особенно сильно это заметно в городах - Санкт-Петербург и Москва. При этом, наиболее востребованными являются должности в сфере IT как наиболее перспективные и востребованные в наше время.  
Интересное наблюдение заключается в том, что всего 11.4% кандидатов из всего датасета рассматривают ту же должность, что и занимают в данный момент. Что косвенно подтверждает предположение о том, что в более раннем возрасте кандидаты чаще меняют должности и сферу деятельности, а с возрастом, набравшись опыта, чаще остаются в одной компании. При этом наибольшее количество совпадений должностей наблюдается в Москве и Санкт-Петербурге.  
А вот соискатели пенсионного возраста встречаются крайне редко, по всей видимости из-за выхода на пенсию, либо карьерного роста внутри компании без размещения резюме или же устройства на работу по рекомендациям и приглашениям других компаний.
3. Анализ кандидатов для заказчиков показывает небольшие по количеству кандидатов выборки, в виду специфичности запросов.  
В первом случае не многие готовы работать вахтовым методом, даже несмотря на более высокие зарплаты и наличие профильных ВУЗов, (которые готовят профильных специалистов для добывающих компаний) в городах, среди которых осуществляется поиск.  
Во втором случае молодых соискателей в Санкт-Петербурге хоть и не так много, зато подавляющее большинство из них желают работать в сфере IT, что в очередной раз подтверждает предположение о том, что направление IT активно развивается и является наиболее востребованным. А также, это свидетельствует о высоком качестве образования по IT специальностям в городе.


[&#8679; к оглавлению](#Оглавление:)